In [38]:
import pandas as pd
import numpy as np
import altair as alt
import os, random
from Bio import SeqIO
import gensim
from gensim.models import Word2Vec
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn import preprocessing, model_selection
import altair as alt


## Defining paths for each and every omic

In [2]:
path_root_data = os.path.join ('..', 'Data', 'Extracted', 'Databases')
path_all_fasta = os.path.join (path_root_data, 'fasta_files', 'AllBins')
path_normalised_metabolomics = os.path.join (path_root_data, 'Metabolomics', 'Normalised_Tables')
path_model_save_root = 'Saved_models'


In [3]:
num_of_mags = len([i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')])
SEED = 42
END = 25
num_of_workers = 3
random.seed(SEED)
np.random.seed(SEED)

---
## MAG-related functions
---

In [36]:
# Function that splits each genome into k-mers thus creating even longer sentence (MAG)
# It returns tokenized genome i.e. [kmer, kmer,...]
def split_genome (genome, k = 5):
    new_genome = []
    n = len(genome)
    
    if n-k <=0:
        return genome
    else:
        for i in range(n-k):
            new_genome.append(genome[i:i+k])
        
        return new_genome


def vectorize_one_mag (one_mag, w2v_model):
    
    # We have to generate vectors for each word in one MAG and then create vector representation of that MAG
    # by averaging vectors of its words
    zero_vector = np.zeros(w2v_model.vector_size)
    word_vectors = []
    one_mag_vector = []
    
    for sentence in one_mag:
        for word in sentence:
            if word in w2v_model.wv:
                try:
                    word_vectors.append(w2v_model.wv[word])
                except KeyError:
                    print ('Key Error')
                    continue
    
    if word_vectors:
        word_vectors = np.asarray(word_vectors)
        one_mag_vector = word_vectors.mean (axis=0)
    
    else:
        one_mag_vector = zero_vector
    
    return one_mag_vector


# Function that vectorizes a MAG (document) with a pretrained word2vec model. It returns vector representation of a given MAG
# Vectorization is done by averaging word (k-mer) vectors for the whole document (MAG)
def vectorize_mags (w2v_model, end = 25):
    
    print ('Vectorizing MAGs')
    
    fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]
    list_of_mag_vectors = []
    
    # This was done so that I could work with first 'end' FASTA files only. Otherwise, I should just remove: i, and enumerate
    for i, fasta_file_name in enumerate(fasta_files):
        
        if i == end:
            break
        
        else:
            with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
                
                one_mag = []
                for fasta_string in SeqIO.parse(input_file, "fasta"):
                    
                    # Get kmers of a genome and create a sentence (list of words)
                    temp_kmers = split_genome (str(fasta_string.seq))
                    
                    # Create a document (list of sentences)
                    one_mag.append(temp_kmers)
                    
                # Vectorizing MAGs one by one
                list_of_mag_vectors.append (vectorize_one_mag (one_mag, w2v_model))
    
    print ('Finished vectorizing')
    
    return list_of_mag_vectors
    
    

# If one wants to import MAGs in order to vectorize them, one should use start argument in order to skip first 'start' MAGs
# If one wants to import MAGs to train word2vec model, one should use only end argument, so that first 'end' MAGs are used for training
# Todo: Implement randomisation in picking MAGs for training, and don't use first 'start' MAGs for training
# (create list of indexes from 0 to 1364, use sklearn split train test, then traverse directory and use only MAGs with indexes in train for training w2v)
def import_mags_and_build_model (end = 25, path_all_fasta = path_all_fasta):
    
    print ('Importing MAGs')
    
    # There are 1364 MAGs enclosed in FASTA files
    # I have to traverse every FASTA file, and in each file every sequence
    
    fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]
    
    # This was done so that I could work with first 100 FASTA files only. Otherwise, I should just remove: i, and enumerate
    for i, fasta_file_name in enumerate(fasta_files):
        
        if i == end:
            break
        
        else:
            with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
                
                one_mag = []
                for fasta_string in SeqIO.parse(input_file, "fasta"):
                    
                    # Get kmers of a genome and create a sentence (list of words)
                    temp_kmers = split_genome (str(fasta_string.seq))
                    
                    # Create a document (list of sentences)
                    one_mag.append(temp_kmers)
                    
                # If we do not have a model, we build one
                if i == 0:
                    print ('Building w2v model')
                    # We build our model on the first MAG
                    w2v_model = Word2Vec (sentences = one_mag, size = 100, workers = num_of_workers, seed=SEED)
                
                # Else we just expand its vocabulary
                else:
                    # Now we expand our vocabulary
                    w2v_model.build_vocab (one_mag, update = True)
                    
    print ('Finished building')
    
    return w2v_model


def train_model (w2v_model, epochs, end = 25):
    
    print ('Starting model training')
    
    # There are 1364 MAGs enclosed in FASTA files
    # I have to traverse every FASTA file, and in each file every sequence
    
    fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]
    
    # This was done so that I could work with first 100 FASTA files only. Otherwise, I should just remove: i, and enumerate
    for i, fasta_file_name in enumerate(fasta_files):
        
        if i == end:
            break
        
        else:
            with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
                
                one_mag = []
                for fasta_string in SeqIO.parse(input_file, "fasta"):
                    
                    # Get kmers of a genome and create a sentence (list of words)
                    temp_kmers = split_genome (str(fasta_string.seq))
                    
                    # Create a document (list of sentences)
                    one_mag.append(temp_kmers)
                    
                
                w2v_model.train (one_mag, total_examples = w2v_model.corpus_count, epochs = epochs)
                    
    print ('Model training finished')
    
    return w2v_model


def visualize_with_pca (data, labels):
    
    pca_model = PCA (n_components = 2, random_state = SEED)
    data_transformed = pca_model.fit_transform (data)
    data_transformed = pd.DataFrame (data_transformed)
    data_transformed.columns = ['PC_1', 'PC_2']
    data_transformed['Labels'] = labels
    
    chart = alt.Chart(data_transformed).mark_circle().encode(
        alt.X ('PC_1:Q'),
        alt.Y ('PC_2:Q'),
        alt.Color ('Labels:N')
    )
    
    return chart



---
## Importing Metabolomic data

---

In [5]:
metabolomics_file_name = os.path.join(path_normalised_metabolomics, os.listdir(path_normalised_metabolomics)[0])
metabolomics_df = pd.read_csv (metabolomics_file_name, delimiter = '\t')
metabolomics_df

,Metabolite,tp,date,type,known_type,means,medians,sds,N,se,ci,type2,measurement,KEGG.Compound.ID,Chebi.Name,Chebi.Name_combined
0,!Phosphoric_acid_3TMS,D1,2012-04-17,snp,known,0.757113,0.737943,0.098650,3,0.056956,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
1,!Phosphoric_acid_3TMS,D10,2012-01-25,snp,known,0.720935,0.873509,0.266593,3,0.153918,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
2,!Phosphoric_acid_3TMS,D11,2012-03-22,snp,known,0.625047,0.652268,0.057820,3,0.033382,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
3,!Phosphoric_acid_3TMS,D12,2012-01-19,snp,known,0.964956,0.941878,0.053754,3,0.031035,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
4,!Phosphoric_acid_3TMS,D13,2011-05-13,snp,known,0.963984,0.974479,0.071659,3,0.041373,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18865,Valine_2TMS,D51,2011-08-11,bp,known,0.704821,0.478251,0.428368,3,0.247319,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine
18866,Valine_2TMS,D6,2011-06-03,bp,known,1.085888,1.197471,0.487484,3,0.281449,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine
18867,Valine_2TMS,D7,2012-04-04,bp,known,2.958023,2.813145,0.341213,3,0.197000,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine
18868,Valine_2TMS,D8,2012-02-08,bp,known,2.515980,2.322248,0.618556,3,0.357123,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine


In [6]:
metabolomics_df['date'] = pd.to_datetime(metabolomics_df['date'])
metabolomics_df.insert (0, 'date', metabolomics_df.pop('date'))
metabolomics_df.sort_values ('date', inplace = True, ignore_index = True)
metabolomics_df

,date,Metabolite,tp,type,known_type,means,medians,sds,N,se,ci,type2,measurement,KEGG.Compound.ID,Chebi.Name,Chebi.Name_combined
0,2011-03-21,Leucine_2TMS,D37,sp,known,1.440122,1.489979,0.146440,3,0.084547,4.302653,extracellular,polar,"C16439,C00123,C01570",leucine;L-leucine;D-leucine,leucine;L-leucine;D-leucine
1,2011-03-21,No match: 1181.78_EM_SP_D1_1_223,D37,sp,unknown,3.778469,3.801417,1.924377,3,1.111040,4.302653,extracellular,polar,NaN,NaN,NaN
2,2011-03-21,unknown#emu_WW_2144.25,D37,bnp,unknown,0.949545,0.947209,0.065422,3,0.037771,4.302653,intracellular,nonpolar,NaN,NaN,NaN
3,2011-03-21,Unknown#bth_pae_001,D37,bp,unknown,2.095116,1.809891,1.916896,3,1.106721,4.302653,intracellular,polar,NaN,NaN,NaN
4,2011-03-21,No match: 2220.34_EM_BP_D1_1_192,D37,bp,unknown,1.631643,1.571125,0.168501,3,0.097284,4.302653,intracellular,polar,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18865,2012-05-03,Octadecanol_1TMS,D48,bnp,known,1.364105,1.457780,0.170311,3,0.098329,4.302653,intracellular,nonpolar,D01924,octadecan-1-ol,octadecan-1-ol
18866,2012-05-03,No match: 1447.93_EM_BP_D2_1_57,D48,bp,unknown,4.815774,1.160165,6.835474,3,3.946463,4.302653,intracellular,polar,NaN,NaN,NaN
18867,2012-05-03,Lactulose_8TMS,D48,bp,known,0.697489,0.684339,0.049632,3,0.028655,4.302653,intracellular,polar,C07064,lactulose,lactulose
18868,2012-05-03,No match: 2004.78_EM_BNP_D10_1_205,D48,bnp,unknown,0.785996,0.744529,0.139901,3,0.080772,4.302653,intracellular,nonpolar,NaN,NaN,NaN


## MAG examination

To do:
1. Make kmers of some length, out of every word in every sentence
2. Encode every sentence with Word2Vec and CBOW (because it's faster than skip-gram)
3. Reduce dimensionality with PCA or with autoencoder
    > Maybe it is good to do the first 3 steps while importing, for each and every sentence. That way, I might use less memory
4. Cluster similar sentences (MAGs) to get rMAGs

> Important: this is not temporal data, since there are no timestamps!


In [7]:

# FOR CLUSTERING I SHOULD CREATE A DATAFRAME WITH MAGs INDEXES AND THEIR VECTOR REPRESENTATIONS

final_model = import_mags_and_build_model (end = END, path_all_fasta = path_all_fasta)


Importing MAGs
Building w2v model
Finished building


In [8]:
final_model.wv.most_similar('ACGGC')

[('CCGGC', 0.6029188632965088),
 ('ACGGA', 0.5602467060089111),
 ('TCGGC', 0.5392060279846191),
 ('GCGGC', 0.5378471612930298),
 ('ACGGT', 0.48065903782844543),
 ('ACGGG', 0.4784856140613556),
 ('CTACG', 0.28459882736206055),
 ('TACGA', 0.2798745036125183),
 ('AACGA', 0.2734222412109375),
 ('ACGTT', 0.273221492767334)]

In [9]:
# Train model. It tooks ~10 minutes for END = 25 amount of MAGs
epochs = 10

final_model = train_model (final_model, epochs = epochs, end = END)


Starting model training
Model training finished


In [10]:
final_model.wv.save_word2vec_format(os.path.join (path_model_save_root, 'model_25.bin'), binary=True) 

Now I should vectorize documents with this model. For further use, I could save this model's weights, and use it to vectorize all mags. That would take a lot, but every MAG will have its vector representation
> This could be done by importing one MAG at a time, then tokenizing it (like before), then getting vector representations of that MAG's sentences (genomes) and then finding the vector representation of the whole MAG (document). If I do that for one MAG at a time, There is no need to worry about memory


In [11]:
list_of_mag_vectors = vectorize_mags (final_model, end = END)

Vectorizing MAGs
Finished vectorizing


In [12]:
list_of_mag_vectors[0:2]

[array([ 0.3447247 , -0.98493254,  0.31138113,  0.9457629 , -0.6776731 ,
        -0.16597524,  0.9591458 ,  2.5192375 ,  0.8832711 , -0.2430107 ,
         1.6672454 , -2.3872983 ,  0.65017176, -0.28312284, -1.4965318 ,
        -1.2259525 ,  0.23684104, -0.35258517,  0.08118253,  0.62865   ,
        -0.17559732, -0.10104723,  1.0475872 ,  0.9804583 ,  1.5938567 ,
        -1.3744875 , -0.5204637 ,  0.01642581,  1.2500559 ,  2.1099281 ,
        -0.49030125,  0.6041928 , -0.87914413,  0.08175804,  1.5866882 ,
        -0.3342121 , -0.83550924, -1.4288213 ,  0.07086587, -1.9811511 ,
        -1.0529662 , -0.990749  ,  2.3318615 , -0.08571481,  1.5839869 ,
         0.05786707,  2.2484546 , -1.2275187 , -0.3562304 ,  1.1188687 ,
        -0.13782707, -0.03078756, -1.6636964 ,  1.1800835 ,  1.4556597 ,
        -1.3416642 , -0.05779186,  1.6490489 , -0.62548006,  1.5076993 ,
         1.0270246 ,  0.828531  , -0.2174203 , -0.28671232, -1.3789287 ,
        -0.3863798 ,  0.354306  , -0.7671363 , -0.5

In [13]:
mags_df = pd.DataFrame (list_of_mag_vectors)
mags_df


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.344725,-0.984933,0.311381,0.945763,-0.677673,-0.165975,0.959146,2.519238,0.883271,-0.243011,...,1.027305,-0.684256,0.842344,0.982793,-1.115196,-1.389488,0.360621,-2.190310,1.044440,0.344591
1,0.086253,0.481529,0.285439,-1.597487,0.084007,0.502726,-1.124469,-2.711813,-0.082691,0.808056,...,0.751947,-0.567273,-0.547687,-0.018143,0.482547,1.292929,-0.480422,2.626672,-1.383779,-0.451343
2,-0.818952,-1.239758,0.806250,1.519856,-0.290418,-0.264609,0.906635,2.390739,-0.411772,-0.718464,...,0.274231,-0.460688,1.217699,0.447533,-0.244538,-2.007320,1.051488,-2.756425,2.020143,0.185595
3,0.087183,0.527681,0.683097,-1.658966,-0.213686,0.772661,-1.532721,-3.115765,-0.120055,1.104269,...,0.539762,-0.430546,-0.241169,-0.086811,0.808656,1.170911,-0.667988,2.736131,-1.349261,-0.597348
4,0.104078,0.523030,0.664976,-1.584116,-0.150568,0.790583,-1.426932,-3.024777,-0.114511,1.100467,...,0.484436,-0.492954,-0.212373,-0.144619,0.706650,1.106278,-0.596556,2.609393,-1.271026,-0.525063
5,0.014663,0.432116,1.100762,-0.587587,-0.268986,0.792427,-1.194816,-2.399201,-0.469793,1.280533,...,0.088165,-0.541312,0.376861,-0.096240,0.815520,0.834893,-0.308112,1.879369,0.026564,-0.450374
6,-0.604413,-1.880543,-0.051779,2.134320,-0.342273,-0.850293,1.257860,4.233354,-0.423073,-1.518789,...,0.403746,-0.517206,0.977598,0.030805,-0.722312,-2.436873,1.935860,-4.078766,1.318376,0.259708
7,-0.466918,-1.944027,0.441384,1.657203,-0.654853,-0.361336,1.080996,3.812093,0.016572,-1.234429,...,0.813006,-0.365175,1.063718,0.750750,-0.798019,-2.832770,1.128418,-3.906541,1.899582,0.186630
8,0.351202,-0.651379,0.771672,0.734120,-0.723834,0.184430,0.033661,0.868674,0.213349,0.448700,...,0.851811,-0.615773,0.845860,0.614016,-0.545951,-0.934811,0.101101,-0.997692,1.270653,0.073550
9,-0.055697,-0.071840,0.714666,0.696301,-0.560185,0.275882,0.003019,0.233549,-0.053218,0.568498,...,0.304297,-0.734093,0.968042,0.301389,0.070306,-0.267304,0.279300,-0.504349,0.932093,0.187632


---
## Data preprocessing
---

In [14]:
mag_scaler = preprocessing.StandardScaler()
scaled_mags_df = mag_scaler.fit_transform(mags_df)

scaled_mags_df

array([[ 1.08036059, -0.79333764, -0.53660622, ..., -0.97476679,
         0.7086667 ,  1.26536613],
       [ 0.24232101,  0.94163843, -0.60586657, ...,  1.03505808,
        -1.35285303, -0.98385411],
       [-2.69261742, -1.09482215,  0.78460615, ..., -1.21097122,
         1.53702388,  0.81606068],
       ...,
       [-1.40785227, -1.72767858, -1.96697477, ..., -1.58552797,
         0.93797092,  1.52880821],
       [ 0.19818926, -0.1341928 ,  2.21882058, ...,  0.41704726,
         0.66681469,  0.45207539],
       [ 0.43035118,  0.99931165,  0.69771964, ...,  1.09983054,
        -1.11663466, -1.96675936]])

---
## Clustering
---

### 1. K-means

In [18]:
k_range_end = 15 # Usually it is sqrt(# of mags)

k_range = range(1, k_range_end)

k_mean_models = [KMeans (n_clusters = i, random_state = SEED) for i in k_range]
k_scores = [k_mean_model.fit(scaled_mags_df).score(scaled_mags_df) for k_mean_model in k_mean_models]
k_data = pd.DataFrame ({'k_range':k_range, 'k_scores':k_scores})
k_data


,k_range,k_scores
0,1,-2500.000000
1,2,-913.125017
2,3,-596.802598
3,4,-476.311370
4,5,-384.997930
5,6,-318.072367
6,7,-259.527094
7,8,-213.584082
8,9,-183.692437
9,10,-161.485470


In [19]:
k_num_chart = alt.Chart(data = k_data).mark_line().encode(
    alt.X ('k_range:Q'),
    alt.Y ('k_scores:Q')
)

k_num_chart

alt.Chart(...)

In [20]:
# We can see from the chart above that 6 or 7 clusters are optimal for this task (where END = 25 MAGs)
num_of_clusters = 7

k_means_model = KMeans (n_clusters = num_of_clusters, random_state = SEED)
k_means_predicted = k_means_model.fit_predict(scaled_mags_df)
k_means_predicted

array([5, 3, 2, 3, 3, 6, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 3, 1, 6, 5, 0, 1,
       2, 4, 6], dtype=int32)

In [37]:
k_means_chart = visualize_with_pca (scaled_mags_df, k_means_predicted)
k_means_chart


alt.Chart(...)

### 2. DBSCAN

In [48]:
MIN_SAMPLES = 3
EPS = 7

dbscan_model = DBSCAN(eps = EPS, min_samples = MIN_SAMPLES, n_jobs = num_of_workers)
dbscan_predicted = dbscan_model.fit_predict (scaled_mags_df)
dbscan_predicted

array([ 0,  1, -1,  1,  1,  1, -1, -1,  0,  0,  0,  0,  0,  1,  1,  1,  1,
        1,  1,  0,  0,  1, -1, -1,  1])

In [53]:
dbscan_core_samples = np.zeros_like(dbscan_predicted, dtype=bool)
dbscan_core_samples[dbscan_model.core_sample_indices_] = True
print(dbscan_core_samples)

[ True  True False  True  True  True False False  True  True  True  True
  True  True  True  True  True  True  True False  True  True False False
  True]


In [ ]:
# Visualize clusters

---
## Evaluation
---